# 概率与统计简介
在本笔记本中，我们将练习一些之前讨论过的概念。许多概率与统计的概念在用于数据处理的主要Python库中都有良好的体现，比如 `numpy` 和 `pandas`。


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## 随机变量和分布
让我们从 0 到 9 的均匀分布中抽取一个包含 30 个值的样本。我们还将计算均值和方差。


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

为了直观估计样本中有多少不同的值，我们可以绘制**直方图**：


In [ ]:
plt.hist(sample)
plt.show()

## 分析真实数据

均值和方差在分析现实世界数据时非常重要。让我们加载关于棒球运动员的数据，来自 [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> 我们这里使用了一个名为[**Pandas**](https://pandas.pydata.org/)的软件包来进行数据分析。我们将在本课程后面讨论更多关于Pandas和在Python中处理数据的内容。

让我们计算年龄、身高和体重的平均值：


In [ ]:
df[['Age','Height','Weight']].mean()

现在让我们关注身高，并计算标准差和方差：


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

除了平均值外，查看中位数和四分位数也是有意义的。它们可以通过**箱线图**来可视化：


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

我们还可以制作数据集子集的箱线图，例如按球员角色分组。


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **注意**：该图表表明，平均来看，一垒手的身高高于二垒手的身高。稍后我们将学习如何更正式地检验这个假设，以及如何证明我们的数据在统计上有显著性以支持该结论。

年龄、身高和体重都是连续随机变量。你认为它们的分布是什么样的？发现答案的一个好方法是绘制数值的直方图：


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 正态分布

让我们创建一个人工权重样本，该样本服从与我们的真实数据相同均值和方差的正态分布：


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

由于现实生活中的大多数数值服从正态分布，因此我们不应该使用均匀随机数生成器来生成样本数据。以下是尝试使用均匀分布（由 `np.random.rand` 生成）生成体重时的情况：


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## 置信区间

现在让我们计算棒球运动员的体重和身高的置信区间。我们将使用[这个stackoverflow讨论](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data)中的代码：


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## 假设检验

让我们探索棒球运动员数据集中不同的角色：


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

让我们来检验一项假设：一垒手比二垒手更高。最简单的方法是检验置信区间：


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

我们可以看到这些区间没有重叠。

一种统计上更正确的验证假设的方法是使用**Student t检验**：


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

`ttest_ind` 函数返回的两个值是：
* p 值可以被视为两个分布具有相同均值的概率。在我们的例子中，它非常低，这意味着有强有力的证据支持一垒手更高。
* t 值是用于 t 检验的标准化均值差的中间值，并且它会与给定置信度的阈值进行比较。


## 使用中心极限定理模拟正态分布

Python 中的伪随机生成器设计为产生均匀分布。如果我们想创建一个正态分布的生成器，可以使用中心极限定理。为了获得一个正态分布的值，我们只需计算一个均匀生成样本的平均值。


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## 相关性与邪恶棒球公司

相关性使我们能够发现数据序列之间的关系。在我们的玩具示例中，假设有一家邪恶的棒球公司根据球员的身高支付薪水——球员越高，拿到的钱越多。假设有一个基本工资为1000美元，并根据身高额外获得0到100美元的奖金。我们将采用MLB的真实球员数据，计算他们的虚拟薪水：


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

现在让我们计算这些序列的协方差和相关系数。`np.cov` 将给我们所谓的**协方差矩阵**，这是协方差在多变量上的扩展。协方差矩阵 $M$ 的元素 $M_{ij}$ 是输入变量 $X_i$ 和 $X_j$ 之间的协方差，主对角线上的值 $M_{ii}$ 是 $X_i$ 的方差。类似地，`np.corrcoef` 会给我们**相关矩阵**。


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

相关系数等于1表示两个变量之间存在强烈的**线性关系**。我们可以通过将一个值与另一个值绘制在图上直观地看到线性关系：


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

让我们看看如果关系不是线性的会发生什么。假设我们的公司决定隐藏身高和薪水之间明显的线性依赖关系，并在公式中引入一些非线性，比如 `sin`：


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

在这种情况下，相关性略小，但仍然相当高。现在，为了让关系更不明显，我们可能想通过向工资中添加一些随机变量来增加额外的随机性。让我们看看会发生什么：


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> 你能猜到为什么这些点会排列成这样的垂直线吗？

我们已经观察到像工资这样的人为设计的概念与观察变量*身高*之间的相关性。让我们也看看两个观察变量，例如身高和体重，是否也相关：


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

不幸的是，我们没有得到任何结果——只有一些奇怪的 `nan` 值。这是因为我们的序列中有些值未定义，表示为 `nan`，这导致运算结果也未定义。通过查看矩阵我们可以看到，`Weight` 是有问题的列，因为计算了 `Height` 值的自相关。

> 这个例子展示了**数据准备**和**清理**的重要性。没有适当的数据，我们无法计算任何东西。

让我们使用 `fillna` 方法填充缺失值，并计算相关性：


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

确实存在相关性，但没有我们人工示例中那么强烈。实际上，如果我们查看一个值与另一个值的散点图，关系会明显不那么明显：


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Conclusion

在本笔记本中，我们学习了如何对数据执行基本操作以计算统计函数。我们现在知道如何使用完善的数学和统计工具来验证某些假设，以及如何根据数据样本计算任意变量的置信区间。


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免责声明**：  
本文件由AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)翻译。我们尽力确保译文的准确性，但请注意自动翻译可能存在错误或不准确之处。原始文件的母语版本应被视为权威来源。对于关键信息，建议采用专业人工翻译。因使用本翻译而产生的任何误解或误释，我们概不负责。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
